# 1. Introduction

## 1.1 Imports
Import libraries here.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from kuberspatiotemporal import CompoundModel, Feature, SpatialModel, KuberModel
from kuberspatiotemporal.tools import make_ellipses

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
import matplotlib.pyplot as plt

# 2. Load Data

In [ ]:
data = pd.read_json('data/spatial_data_lisboa.json')

In [ ]:
data.head(2)

In [ ]:
# Filter office and home locations
data = data[['latitude', 'longitude', 'timestamp']]
data = data[(
    (((data['latitude']>38.710104) & (data['latitude']<38.710648)) & ((data['longitude']>-9.139794) & (data['longitude']<-9.1389)))
 | (((data['latitude']>38.725828) & (data['latitude']<38.726512)) & ((data['longitude']>-9.133944) & (data['longitude']<-9.133169))))]
data['time'] = [ts.hour + ts.minute/60 + ts.second/3600 for ts in data.timestamp]
data['weekday'] = [ts.dayofweek for ts in data.timestamp]

In [ ]:
display(data, data.hist(column="weekday", bins=6))

# 3. Learn Spatiotemporal Model - 4D

In [ ]:
limits_ = [np.min(data[['latitude', 'longitude', 'time']].values, axis=0),np.max(data[['latitude', 'longitude', 'time']].values, axis=0)]

In [ ]:
kst = CompoundModel(
    n_dim=4,
    n_iterations=200,
    scaling_parameter=1.1,
    nonparametric=True,
    online_learning=False,
    loa=True,
    features=[
        Feature(SpatialModel(n_dim=3, min_eigval=1e-9, limits=limits_), [0, 1, 2]),
        Feature(KuberModel(n_symbols=7), [3])
    ],
)

In [ ]:
pipeline = make_pipeline(
    make_column_transformer(
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "latitude"),
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "longitude"),
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "time"),
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "weekday"),
    ),
    kst,
)

In [ ]:
pipeline.fit(data[['latitude', 'longitude', 'time', 'weekday']])

In [ ]:
kst.rvs(1000)

## 3.1 Binary score

In [ ]:
kst.loa = False
kst.score_threshold = kst.get_score_threshold(data[['latitude', 'longitude', 'time', 'weekday']].values, lower_quantile=0, upper_quantile=1)
kst.score(np.array([[38.7104174,-9.1417113,16,4]]))

## 3.2 Compute score given quantiles

In [ ]:
kst.loa = False
kst.score_threshold = None
kst.quantiles = kst.get_score_threshold(data[['latitude', 'longitude', 'time', 'weekday']].values, lower_quantile=0, upper_quantile=0.3)
kst.score(np.array([[38.7104174,-9.1417113,16,4]]))

## 3.3 Compute score given box

In [ ]:
kst.loa = True
kst.features[0].model.box=[0.005,0.005,1]
kst.score(np.array([[38.7104174,-9.1417113,16,4]]))

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker, Rectangle

m = Map(center=(np.mean(data.latitude), np.mean(data.longitude)), zoom=10)

colors = np.array(['red', 'green'])
for lat, lon, label in zip(data.latitude.values, data.longitude.values, pipeline.score_samples(data[['latitude', 'longitude', 'time', 'weekday']])):
    circle_marker = CircleMarker()
    circle_marker.location = (lat, lon)
    circle_marker.radius = 3
    circle_marker.color = colors[int(label)]
    circle_marker.fill_color = colors[int(label)]

    m.add_layer(circle_marker)

m

# 4 Spatio Model

In [ ]:
limits = np.array([[data.latitude.min(), data.longitude.min()],[data.latitude.max(), data.longitude.max()]])
limits = limits + np.array([[-0.01], [0.01]])

In [ ]:
spatio_model = SpatialModel(n_dim=2, min_eigval=1e-10, nonparametric=True, n_iterations=10, limits=limits,
                    scaling_parameter=0.01, loa=True, decay=1, n_components=2)

In [ ]:
spatio_model.fit(data[['latitude', 'longitude']].values)

In [ ]:
from kuberspatiotemporal.tools import make_ellipses
f, ((ax1, ax2)) = plt.subplots(1,2, figsize=(18, 10))
ax1.scatter(data.latitude, data.longitude, marker='o',
            s=25, edgecolor='k')
ax1.set_title('data')

make_ellipses(spatio_model, ax2)
ax2.scatter(data.latitude, data.longitude,
            s=25, edgecolor='k')
ax2.set_title('ellipses')

In [ ]:
spatio_model.rvs(10)

## 4.1 Binary score

In [ ]:
spatio_model.get_score_threshold(data[['latitude', 'longitude']].values, lower_quantile=0, upper_quantile=0.95)

In [ ]:
spatio_model.loa = False
spatio_model.score_threshold = spatio_model.get_score_threshold(data[['latitude', 'longitude']].values, lower_quantile=0, upper_quantile=1)
spatio_model.score(np.array([[38.7104174,-9.1417113]]))

## 4.2 Compute score given quantiles

In [ ]:
spatio_model.loa = False
spatio_model.score_threshold = None
spatio_model.quantiles = spatio_model.get_score_threshold(data[['latitude', 'longitude']].values, lower_quantile=0, upper_quantile=0.3)
spatio_model.score(np.array([[38.7104174,-9.1417113]]))

## 4.3 Compute score given box

In [ ]:
spatio_model.loa = True
spatio_model.box=[0.05,0.05]
spatio_model.score(np.array([[38.7104174,-9.1417113]]))

# 5 Spatiotemporal Model

In [ ]:
spatiotemporal_model = SpatialModel(n_dim=3, min_eigval=1e-9, limits=limits_, n_iterations=200, 
                                    scaling_parameter=1.1, nonparametric=True, online_learning=False,
                                    loa=True)

In [ ]:
spatiotemporal_model.fit(data[['latitude', 'longitude', 'time']].values)

In [ ]:
spatiotemporal_model.rvs(10)

## 5.1 Binary score

In [ ]:
spatiotemporal_model.get_score_threshold(data[['latitude', 'longitude','time']].values, lower_quantile=0, upper_quantile=0.95)

In [ ]:
spatiotemporal_model.loa = False
spatiotemporal_model.score_threshold = spatiotemporal_model.get_score_threshold(data[['latitude', 'longitude', 'time']].values, lower_quantile=0, upper_quantile=1)
spatiotemporal_model.score(np.array([[38.7104174,-9.1417113, 12]]))

## 5.2 Compute score given quantiles

In [ ]:
spatiotemporal_model.loa = False
spatiotemporal_model.score_threshold = None
spatiotemporal_model.quantiles = spatiotemporal_model.get_score_threshold(data[['latitude', 'longitude','time']].values, lower_quantile=0, upper_quantile=0.3)
spatiotemporal_model.score(np.array([[38.7104174,-9.1417113,10]]))

## 4.3 Compute score given box

In [ ]:
spatiotemporal_model.loa = True
spatiotemporal_model.box=[0.05,0.05,1]
spatiotemporal_model.score(np.array([[38.7104174,-9.1417113,10]]))

# 6 Weekday Model

In [ ]:
weekday_model = KuberModel(n_symbols=7, n_components=7, loa=True)

In [ ]:
weekday_model.fit(data[['weekday']].values)

In [ ]:
weekday_model.rvs(100)

## 6.1 Binary score

In [ ]:
weekday_model.get_score_threshold(data[['weekday']].values, lower_quantile=0, upper_quantile=0.95)

In [ ]:
weekday_model.loa = False
weekday_model.score_threshold = weekday_model.get_score_threshold(data[['weekday']].values, lower_quantile=0, upper_quantile=0.95)
weekday_model.score(np.array([[6]]))

## 6.2 Compute score given quantiles

In [ ]:
weekday_model.loa = False
weekday_model.score_threshold = None
weekday_model.quantiles = weekday_model.get_score_threshold(data[['weekday']].values, lower_quantile=0, upper_quantile=0.9)
weekday_model.score(np.array([[4]]))

## 4.3 Compute score given box

In [ ]:
weekday_model.loa = True
weekday_model.box=[1]
weekday_model.score(np.array([[3]]))